our choice of output function depends on two primary factors about what you are trying to predict: its shape and its range.

This means that our output function is determined by what we're trying to do: classification or regression.

Common output functions include:

Sigmoid for binary classification -> value between 0 and 1
Softmax for multi-class classification -> multiclass cross entrophy
Identity or ReLU for regression -> mean squared error

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [ ]:
# Establish our transform
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load train and test datasets
training_data = datasets.CIFAR10(root="data", train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root="data", train=False, download=True, transform=transform)

# Create the training and test dataloaders with a batch size of 32
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

In [ ]:
# This code is provided for you. 
# Feel free to look it over but if you modify it, it may break!
def train_network_classification(net, train_loader, test_loader):
    num_epochs = 10
    
    # We'll use Negative Log Likelihood Loss as our objective function here. Leave it fixed for now.
    criterion = nn.NLLLoss()

    # Don't worry about the choice of optimizer here. Leave it fixed for now.
    optimizer = optim.SGD(mlp.parameters(), lr=0.005, momentum=0.9)
    
    # Establish a list for our history
    train_loss_history = list()
    val_loss_history = list()

    for epoch in range(num_epochs):
        net.train()
        train_loss = 0.0
        train_correct = 0
        for i, data in enumerate(train_loader):
            # data is a list of [inputs, labels]
            inputs, labels = data
            
            # Pass to GPU if available.
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, preds = torch.max(outputs.data, 1)
            train_correct += (preds == labels).sum().item()
            train_loss += loss.item()
        print(f'Epoch {epoch + 1} training accuracy: {train_correct/len(train_loader):.2f}% training loss: {train_loss/len(train_loader):.5f}')
        train_loss_history.append(train_loss)

        
        val_loss = 0.0
        val_correct = 0
        net.eval()
        for inputs, labels in test_loader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()
            
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            
            _, preds = torch.max(outputs.data, 1)
            val_correct += (preds == labels).sum().item()
            val_loss += loss.item()
        print(f'Epoch {epoch + 1} validation accuracy: {val_correct/len(test_loader):.2f}% validation loss: {val_loss/len(test_loader):.5f}')
        val_loss_history.append(val_loss)           

    plt.plot(train_loss_history, label="Training Loss")
    plt.plot(val_loss_history, label="Validation Loss")
    plt.legend()
    plt.show()

def train_network_regression(net, train_loader, test_loader):
    num_epochs = 10
    
    criterion = nn.L1Loss(reduction='sum')

    # Don't worry about the choice of optimizer here. Leave it fixed for now.
    optimizer = optim.SGD(mlp.parameters(), lr=0.05)
    
    # Establish a list for our history
    train_loss_history = list()
    val_loss_history = list()

    for epoch in range(num_epochs):
        net.train()
        train_loss = 0.0
        for i, data in enumerate(train_loader):
            # data is a list of [inputs, labels]
            inputs, labels = data
            
            # Pass to GPU if available.
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        print(f'Epoch {epoch + 1} training loss: {train_loss/len(train_loader):.5f}')
        train_loss_history.append(train_loss)

        
        val_loss = 0.0
        net.eval()
        for inputs, labels in test_loader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()
            
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
        print(f'Epoch {epoch + 1} validation loss: {val_loss/len(test_loader):.5f}')
        val_loss_history.append(val_loss)           

    plt.plot(train_loss_history, label="Training Loss")
    plt.plot(val_loss_history, label="Validation Loss")
    plt.legend()
    plt.show()

In [ ]:
class CIFAR_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = F.relu
        self.output = F.log_softmax
        self.fc1 = nn.Linear(32 * 32 * 3, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.output(self.fc3(x), dim=1) # For some output functions, may need to specify dimension.
        return x

# Do not change the name of your model or later cells may fail!
mlp = CIFAR_MLP()
if torch.cuda.is_available():
    mlp.cuda()

In [ ]:
# Now let's train our network!
train_network_classification(mlp, train_loader, test_loader)

In [ ]:
# Load our data and split it into train and test sets
data, target = fetch_california_housing(return_X_y=True)
train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.3)

# Since we are using PyTorch, we need tensors!
train_x = torch.tensor(train_x, dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

# Then we convert those tensors to a TensorDataset
train_california = torch.utils.data.TensorDataset(train_x, train_y)
test_california = torch.utils.data.TensorDataset(test_x, test_y)

# And create our DataLoaders!
train_loader = DataLoader(train_california, batch_size=10, shuffle=True)
test_loader = DataLoader(test_california, batch_size=10)

In [ ]:
class Housing_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = F.relu
        self.hidden = nn.Linear(8, 2)
        self.prediction = nn.Linear(2, 1)

    def forward(self, x):
        x = self.activation(self.hidden(x))
        x = self.prediction(x)

        return x

# Do not change the name of your model or later cells may fail!
mlp = Housing_MLP()
if torch.cuda.is_available():
    mlp.cuda()